In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge

In [2]:
assets = ['b', 's'] # buying and selling assets
base_symbols = ['s', 'l', 'w', 'j', 'e', 'Delta', 'a'] 
# spot price, liquidity, weight, jump size, exponent, delta, anchor price
all_symbols = {}
for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        if base == 'e':
            temp_dict[f"{base}_{asset}"] = symbols(f"{base}_{asset}", integer=True)
        else:
            temp_dict[f"{base}_{asset}"] = symbols(f"{base}_{asset}", positive=True, integer=True)
    all_symbols[asset] = tuple(temp_dict.values())

all_symbols

{'b': (s_b, l_b, w_b, j_b, e_b, Delta_b, a_b),
 's': (s_s, l_s, w_s, j_s, e_s, Delta_s, a_s)}

In [3]:
e_b = all_symbols['b'][base_symbols.index('e')]
e_s = all_symbols['s'][base_symbols.index('e')]
Delta_b = all_symbols['b'][base_symbols.index('Delta')]
Delta_s = all_symbols['s'][base_symbols.index('Delta')]

In [4]:
def deltaBySpot_(s, l, w, j, e, Delta, a):
  return simplify((s - l * w) / w)

deltaBySpot = {asset: deltaBySpot_(*all_symbols[asset]) for asset in assets}
deltaBySpot['b'] = -deltaBySpot['b']
deltaBySpot['b']

l_b - s_b/w_b

In [5]:
deltaBySpot['s']

-l_s + s_s/w_s

In [6]:
def spotByExp_(s, l, w, j, e, Delta, a):
  return simplify(a * ((1 + 1/j) ** e)
)
spotByExp = {asset: spotByExp_(*all_symbols[asset]) for asset in assets}
spotByExp['b']

a_b*((j_b + 1)/j_b)**e_b

In [7]:
def deltaByExp_(asset, s, l, w, j, e, Delta, a):
    return simplify(deltaBySpot[asset].subs(s, spotByExp[asset]))

deltaByExp = {asset: deltaByExp_(asset, *all_symbols[asset]) for asset in assets}
deltaByExp['b']

-a_b*j_b**(-e_b)*(j_b + 1)**e_b/w_b + l_b

In [8]:
deltaByExp['s']

a_s*j_s**(-e_s)*(j_s + 1)**e_s/w_s - l_s

In [9]:
def expBound_(asset, s, l, w, j, e, Delta, a):
    limit = simplify(log(w * l /a, 1 + 1/j))
    if asset == 'b':
        return Ge(limit - e, 0)
    else:
        return Ge(e, limit)

expBound = {asset: expBound_(asset, *all_symbols[asset]) for asset in assets}
expBound['b']

-e_b + log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b))) >= 0

In [10]:
expBound['s']

e_s >= log((l_s*w_s/a_s)**(1/log((j_s + 1)/j_s)))

In [11]:
def deltaLimit_(asset, s, l, w, j, e, Delta, a):
    e = all_symbols[asset][base_symbols.index('e')]
    return simplify(solveset(deltaByExp[asset] >= 0, e, domain=S.Integers).condition)

deltaLimit = {asset: deltaLimit_(asset, *all_symbols[asset]) for asset in assets}
deltaLimit['b']

a_b*j_b**(-e_b)*(j_b + 1)**e_b/w_b - l_b <= 0

In [12]:
deltaLimit['s']

a_s*j_s**(-e_s)*(j_s + 1)**e_s/w_s - l_s >= 0

In [ ]:
ebUpper = min(deltaLimit['b'].condition.lhs)

In [ ]:
eff = Delta_s / Delta_b
eff

In [ ]:
deltaByExp_b = deltaByExp['b']
deltaByExp_s = deltaByExp['s']
effByExps = eff.subs({Delta_b: deltaByExp_b, Delta_s: deltaByExp_s})
effByExps

In [ ]:
e_b = all_symbols['b'][base_symbols.index('e')]
e_s = all_symbols['s'][base_symbols.index('e')]
H = hessian(effByExps, (e_b, e_s))
H

In [ ]:
# checking for convexity. Both need to be nonnegative
topLeft = simplify(H[0, 0])
#det = simplify(H.det())

topLeft


In [ ]:
nonneg_topLeft_eb = solveset(topLeft >= 0, e_b, domain=S.Integers)
nonneg_topLeft_eb

In [ ]:
nonneg_topLeft_es = solveset(topLeft >= 0, e_s, domain=S.Integers)
nonneg_topLeft_es

In [ ]:
det = H.det()
det

In [ ]:
det = simplify(det)
det

In [ ]:
nonneg_det_eb = solveset(det >= 0, e_b, domain=S.Integers)
nonneg_det_eb

In [ ]:
nonneg_det_es = solveset(det >= 0, e_s, domain=S.Integers)
nonneg_det_es

In [ ]:
nonneg_topLeft = simplify(nonneg_topLeft_eb.condition)
nonneg_topLeft

In [ ]:
simplify(nonneg_topLeft_eb.condition.lhs - nonneg_topLeft_es.condition.lhs)

In [ ]:
nonneg_det = simplify(nonneg_det_eb.condition)
nonneg_det

In [ ]:
simplify(nonneg_det_eb.condition.lhs - nonneg_det_es.condition.lhs)

In [ ]:
simplify(And(expBound['b'] >= e_b, nonneg_topLeft_eb.condition))